<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/gemini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install -q -U google-generativeai

In [2]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
import pandas as pd
import time

In [4]:
model = genai.GenerativeModel('gemini-pro')

In [5]:
def detect(bodies):
    try:
        # Construct prompts for the batch
        prompts = [f"SMS Body: {body}\nIs this message real or fake?" for body in bodies]
        results = []

        # Apply prompts to model in a single batch request
        responses = model.generate_content(prompts)  # Assuming the model can handle a list of prompts

        # Process each response to determine authenticity
        for response in responses:
            if "real" in response.text.lower():
                results.append("Real")
            elif "fake" in response.text.lower():
                results.append("Fake")
            else:
                results.append("Undetermined")

        return results

    except Exception as error:
        print("Error occurred:", str(error))
        return ["Error"] * len(bodies)

In [6]:
from google.colab import files
uploaded = files.upload()

Saving smishTank_set.xlsx to smishTank_set (6).xlsx


In [7]:
# Process excel file
def process_excel(input_file):
    # Assign file to pandas dataframe
    df = pd.read_excel(input_file, header=None)
    # Rename columns
    df.columns = ['SMS Body']

    # To store results for each value
    authenticity_results = []

    # Process rows in batches of 5
    batch_size = 5

    for i in range(0, len(df), batch_size):
        batch = df['SMS Body'][i:i+batch_size].tolist()
        batch_results = [detect(message) for message in batch]
        authenticity_results.extend(batch_results)

        # Print status after processing each batch
        print(f"Processed inputs {i + 1} to {i + len(batch)}, Results = {batch_results}")

        # Add a delay between batches to avoid rate limiting
        time.sleep(20)

    # Assign results to new column
    df['Authenticity'] = authenticity_results

    # Write updated dataframe to new file
    output_file = "processed_" + input_file
    df.to_excel(output_file, index=False)

    # Download modified file to local machine
    files.download(output_file)

In [8]:
#process file
pFile = next(iter(uploaded))
process_excel(pFile)

Processed inputs 1 to 5, Results = [['Real'], ['Fake'], ['Real'], ['Real'], ['Real']]
Processed inputs 6 to 10, Results = [['Real'], ['Real'], ['Fake'], ['Real'], ['Fake']]
Processed inputs 11 to 15, Results = [['Real'], ['Real'], ['Real'], ['Real'], ['Real']]
Processed inputs 16 to 20, Results = [['Fake'], ['Fake'], ['Real'], ['Fake'], ['Real']]
Processed inputs 21 to 25, Results = [['Real'], ['Fake'], ['Fake'], ['Fake'], ['Real']]
Processed inputs 26 to 30, Results = [['Real'], ['Real'], ['Fake'], ['Fake'], ['Real']]
Processed inputs 31 to 35, Results = [['Fake'], ['Real'], ['Real'], ['Real'], ['Real']]
Processed inputs 36 to 40, Results = [['Real'], ['Real'], ['Fake'], ['Fake'], ['Fake']]
Processed inputs 41 to 45, Results = [['Real'], ['Real'], ['Undetermined'], ['Real'], ['Real']]
Processed inputs 46 to 50, Results = [['Real'], ['Real'], ['Real'], ['Real'], ['Real']]
Processed inputs 51 to 55, Results = [['Real'], ['Real'], ['Fake'], ['Real'], ['Fake']]
Processed inputs 56 to 60, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
#download modified file to local machine
files.download("processed_" + pFile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>